# Downloading Titanic dataset
In this project we will use the [Titanic Dataset](https://www.kaggle.com/c/titanic/data) from [Kaggle](https://www.kaggle.com), which contains information about the passengers aboard the Titanic.

![titanic](https://storage.googleapis.com/kaggle-datasets-images/1680488/2754327/6d6c9d987d2c5c5c8ee5b5886d3bbfd3/dataset-cover.jpeg?t=2021-10-28-20-33-47![image.png](attachment:image.png))

# PostgreSQL

## Creating the database and tables
Let's create a `database` called **titanic** and a `table` called **passengers**.

* **NOTE:** You need to install psycopg2 library using **`!pip install psycopg2`** or **`!pip install psycopg2-binary`** if you haven't already.

In [1]:
import psycopg2

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(
    dbname='postgres', #postgres
    user='postgres',
    password='my_password', 
    host='localhost',
    port='5432'
)

# Create a cursor object
cur = conn.cursor()

#conn.autocommit = True

# Create the titanic database
cur.execute('titanic') 

# Close the cursor and connection to the default database
cur.close()
conn.close()

# Connect to the titanic database
conn = psycopg2.connect(
    dbname='titanic',
    user='postgres',
    password='my_password', 
    host='localhost',
    port='5432'
)

# Create a cursor object
cur = conn.cursor()

# Create the passengers table
cur.execute("""
     CREATE TABLE IF NOT EXISTS passengers(
        passenger_id SERIAL PRIMARY KEY, 
    survived BOOLEAN, 
    pclass INTEGER,
    sex VARCHAR(10),
    age FLOAT, 
    sibsp INTEGER, 
    parch INTEGER, 
    fare FLOAT, 
    embarked VARCHAR(10), 
    class VARCHAR(10), 
    who VARCHAR(10), 
    adult_male VARCHAR(10), 
    deck VARCHAR(10), 
    embark_town VARCHAR(255), 
    alive VARCHAR(10), 
    alone VARCHAR(10))
""")

# Commit the changes
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()

## Importing data into the tables
We will use `pandas` to read the `CSV` file and insert the data into the tables.

* **NOTE:** You need to install pandas library using **`!pip install pandas`** if you haven't already.

In [2]:
import pandas as pd

df = pd.read_csv('titanic_data.csv')
print(df)

     survived  pclass     sex   age  sibsp  parch     fare embarked   class  \
0           0       3    male  22.0      1      0   7.2500        S   Third   
1           1       1  female  38.0      1      0  71.2833        C   First   
2           1       3  female  26.0      0      0   7.9250        S   Third   
3           1       1  female  35.0      1      0  53.1000        S   First   
4           0       3    male  35.0      0      0   8.0500        S   Third   
..        ...     ...     ...   ...    ...    ...      ...      ...     ...   
884         0       2    male  27.0      0      0  13.0000        S  Second   
885         1       1  female  19.0      0      0  30.0000        S   First   
886         0       3  female   NaN      1      2  23.4500        S   Third   
887         1       1    male  26.0      0      0  30.0000        C   First   
888         0       3    male  32.0      0      0   7.7500        Q   Third   

       who  adult_male deck  embark_town alive  alo

In [3]:
df =  df.fillna(method="ffill")
df = df.dropna()
df.to_csv('titanic_filled.csv', index=False)

In [4]:
df.iloc[:]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,C,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,C,Southampton,no,True
5,0,3,male,35.0,0,0,8.4583,Q,Third,man,True,C,Queenstown,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,0,2,male,27.0,0,0,13.0000,S,Second,man,True,C,Southampton,no,True
885,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
886,0,3,female,19.0,1,2,23.4500,S,Third,woman,False,B,Southampton,no,False
887,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [7]:
# Create a cursor object
cur = conn.cursor()

# Insert each row of data into the passengers table
for index, row in df.iterrows():
    cur.execute('INSERT INTO passengers (survived, pclass, sex, age, sibsp, parch, fare, embarked, class, who, adult_male, deck, embark_town, alive, alone) '
                'VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)', (
                    row['survived'],
                    row['pclass'],
                    row['sex'],
                    row['age'],
                    row['sibsp'], #siblings/Spouses Aboard
                    row['parch'], #parents/Children Aboard
                    row['fare'],
                    row['embarked'],
                    row['class'],
                    row['who'],
                    row['adult_male'],
                    row['deck'],
                    row['deck'],
                    row['embark_town'],
                    row['alive'],
                    row['alone']
                ))

# Commit the changes
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()

survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone


## Executing SQL queries
let's execute some interesting SQL queries on our titanic database.

### 1. Get the number of passengers in each class

In [8]:
# Get the number of passengers in each class
cur.execute('SELECT class, COUNT(*) FROM passengers GROUP BY class')
results = cur.fetchall()
print('\nNumber of passengers in each class: \n')
for result in results:
    print(f'{result[0]} Class: {result[1]}', '\n')
    
# Close the cursor and connection
cur.close()
conn.close()


Number of passengers in each class: 

First Class: 216 

Second Class: 184 

Third Class: 487 



### 2. Get the average age of male and female passengers

In [9]:
# Get the average age of male and female passengers
cur.execute('SELECT sex, AVG(age) FROM passengers GROUP BY sex')
results = cur.fetchall()
print('\nAverage age of male and female passengers:', '\n')
for result in results:
    print(f'{result[0]}: {result[1]:.2f}', '\n')

# Close the cursor and connection
cur.close()
conn.close()


Average age of male and female passengers:  

female: 27.72 

male: 30.43 



### 3. Get the survival rate for each class

In [10]:
# Get the survival rate for each class
cur.execute('SELECT class, AVG(survived::int) FROM passengers GROUP BY class')
results = cur.fetchall()
print('\nSurvival rate for each class:', '\n')
for result in results:
    print(f'{result[0]} Class: {result[1]:.2%}', ''\n')

# Close the cursor and connection
cur.close()
conn.close()


Survival rate for each class:  

First Class: 62.96%

Second Class: 47.28% 

Third Class: 24.24% 



### 4. Get the top 5 passengers (Man or Woman) with the highest fares

In [11]:
# Get the top 5 passengers with the highest fares
cur.execute('SELECT who, fare FROM passengers ORDER BY fare DESC LIMIT 5')
results = cur.fetchall()
print('\nPassengers with the highest fares:', '\n')
for result in results:
    print(f'{result[0]}: ${result[1]:.2f}', '\n')

# Close the cursor and connection
cur.close()
conn.close()


Passengers with the highest fares:  

Man: $512.33 

Man: $263.00 

Man: $263.00 

Woman: $262.38 

Man: $262.38 



### 5. Get the age and class of all female survivors under the age of 18

In [12]:
# Get the age and class of all female survivors under the age of 18
cur.execute('SELECT age, pclass FROM passengers WHERE sex = \'female\' AND survived = TRUE AND age < 18')
results = cur.fetchall()
print('\nFemale survivors under the age of 18:', '\n')
for result in results:
    print(f'Age {float(result[0])}, Class {result[1]:.2f}')

# Close the cursor and connection
cur.close()
conn.close()


Female survivors under the age of 18: 

Age 14.50, Class 2 

Age 14.50, Class 2 

Age 3.00, Class 3 

Age 5.00, Class 3 

Age 0.75, Class 1 

Age 4.00, Class 1 

Age 0.75, Class 1 

Age 5.00, Class 1 

Age 1.00, Class 2 

Age 0.75, Class 1 

Age 4.00, Class 1 

Age 16.00, Class 3 

Age 1.00, Class 2 

Age 16.00, Class 1 

Age 22.00, Class 2 

Age 14.00, Class 2 



# MongoDB

## Create a database and collection in MongoDB

In this section we need to create a **`MongoDB`** database and collection for the titanic dataset using Python.

* **NOTE:** Install and import the **`pymongo`** library using **`!pip install pymongo`**, to connect to the MongoDB database, create a new database, and create a new collection:

In [13]:
import pandas as pd

df = pd.read_csv('titanic_data.csv')
print(df)

     survived  pclass     sex   age  sibsp  parch     fare embarked   class  \
0           0       3    male  22.0      1      0   7.2500        S   Third   
1           1       1  female  38.0      1      0  71.2833        C   First   
2           1       3  female  26.0      0      0   7.9250        S   Third   
3           1       1  female  35.0      1      0  53.1000        S   First   
4           0       3    male  35.0      0      0   8.0500        S   Third   
..        ...     ...     ...   ...    ...    ...      ...      ...     ...   
884         0       2    male  27.0      0      0  13.0000        S  Second   
885         1       1  female  19.0      0      0  30.0000        S   First   
886         0       3  female   NaN      1      2  23.4500        S   Third   
887         1       1    male  26.0      0      0  30.0000        C   First   
888         0       3    male  32.0      0      0   7.7500        Q   Third   

       who  adult_male deck  embark_town alive  alo

In [14]:
df =  df.fillna(method="ffill")
df = df.dropna()
df.to_csv('titanic_filled.csv', index=False)

In [15]:
df.iloc[:]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,C,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,C,Southampton,no,True
5,0,3,male,35.0,0,0,8.4583,Q,Third,man,True,C,Queenstown,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,0,2,male,27.0,0,0,13.0000,S,Second,man,True,C,Southampton,no,True
885,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
886,0,3,female,19.0,1,2,23.4500,S,Third,woman,False,B,Southampton,no,False
887,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [16]:
from pymongo import MongoClient

# Connect to the local MongoDB server
client = MongoClient()

In [17]:
# Create a new database called "titanic"
db = client['titanic']
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'titanic')

In [18]:
# Create a new collection called "passengers"
collection = db['passengers']
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'titanic'), 'passengers')

## Importing data into the tables

Now that we've created our database and collection, we're ready to insert the data from our dataframe into the collection. We can do this using the `insert_many` method:

In [19]:
# Import the titanic data from the CSV file
import csv

# Convert the dataframe to a list of dictionaries
with open('titanic_filled.csv', 'r') as file:
    reader = csv.DictReader(file)
    data = []
    for row in reader:
        data.append(row)
print(data)

# Insert the data into the MongoDB collection
collection.insert_many(data)

# Close the connection to the MongoDB database
client.close()

[{'survived': '1', 'pclass': '1', 'sex': 'female', 'age': '38.0', 'sibsp': '1', 'parch': '0', 'fare': '71.2833', 'embarked': 'C', 'class': 'First', 'who': 'woman', 'adult_male': 'False', 'deck': 'C', 'embark_town': 'Cherbourg', 'alive': 'yes', 'alone': 'False'}, {'survived': '1', 'pclass': '3', 'sex': 'female', 'age': '26.0', 'sibsp': '0', 'parch': '0', 'fare': '7.925', 'embarked': 'S', 'class': 'Third', 'who': 'woman', 'adult_male': 'False', 'deck': 'C', 'embark_town': 'Southampton', 'alive': 'yes', 'alone': 'True'}, {'survived': '1', 'pclass': '1', 'sex': 'female', 'age': '35.0', 'sibsp': '1', 'parch': '0', 'fare': '53.1', 'embarked': 'S', 'class': 'First', 'who': 'woman', 'adult_male': 'False', 'deck': 'C', 'embark_town': 'Southampton', 'alive': 'yes', 'alone': 'False'}, {'survived': '0', 'pclass': '3', 'sex': 'male', 'age': '35.0', 'sibsp': '0', 'parch': '0', 'fare': '8.05', 'embarked': 'S', 'class': 'Third', 'who': 'man', 'adult_male': 'True', 'deck': 'C', 'embark_town': 'Southamp

## Automate data access by connecting the MongoDB database with API

To connect our MongoDB database with an API, we'll use the `Flask web` framework.

In this section we use the `Flask` framework to define five API endpoints for `retrieving`, `creating`, `updating`, and `deleting` data in a **MongoDB** database. For example, in the first part, we create an API that returns all the passengers in the Titanic dataset.

In [20]:
from flask import Flask, jsonify
from pymongo import MongoClient

# Connect to the local MongoDB server
client = MongoClient()

# define MongoDB database and collection
db = client['titanic']
collection = db['passengers']

# create Flask app
app = Flask(__name__)

# create API route to retrieve data from MongoDB collection
@app.route('/passengers', methods=['GET'])
def get_passengers():
    passengers = list(collection.find())
    return jsonify(passengers)

if __name__ == '__main__':
    app.run()
    
# Create five API endpoints
# create API route to:
@app.route('/passengers', methods=['GET'])
def get_passengers():
    # Retrieve all documents from the MongoDB collection
    data = []
    try:
        cursor = collection.find()
        for doc in cursor:
            data.append(doc)
        return jsonify(data), 200
    except errors.PyMongoError as e:
        return jsonify({'error': str(e)}), 500

@app.route('/data/<id>', methods=['GET'])
def get_passengers_by_id(id):
    # Retrieve a specific document from the MongoDB collection by ID
    try:
        doc = collection.find_one({'_id': id})
        if doc:
            return jsonify(doc), 200
        else:
            return jsonify({'error': 'Document not found'}), 404
    except errors.PyMongoError as e:
        return jsonify({'error': str(e)}), 500

@app.route('/data', methods=['POST'])
def create_passengers():
    # Insert a new document into the MongoDB collection
    try:
        payload = request.get_json()
        result = collection.insert_one(payload)
        return jsonify({'inserted_id': str(result.inserted_id)}), 201
    except errors.PyMongoError as e:
        return jsonify({'error': str(e)}), 500

@app.route('/data/<id>', methods=['PUT'])
def update_passengers(id):
    # Update a specific document in the MongoDB collection by ID
    try:
        payload = request.get_json()
        result = collection.update_one({'_id': id}, {'$set': payload})
        if result.modified_count == 1:
            return jsonify({'message': 'Document updated successfully'}), 200
        else:
            return jsonify({'error': 'Document not found'}), 404
    except errors.PyMongoError as e:
        return jsonify({'error': str(e)}), 500

@app.route('/data/<id>', methods=['DELETE'])
def delete_passengers(id):
    # Delete a specific document from the MongoDB collection by ID
    try:
        result = collection.delete_one({'_id': id})
        if result.deleted_count == 1:
            return jsonify({'message': 'Document deleted successfully'}), 200
        else:
            return jsonify({'error': 'Document not found'}), 404
    except errors.PyMongoError as e:
        return jsonify({'error': str(e)}), 500

# Run the Flask app
if __name__ == '__main__':
    app.run(debug=True)

The `MongoClient` class from the pymongo library is used to **establish a connection to the database**, and the `collection` variable is used to **reference the collection to be accessed**.

* **NOTE:** To test the API, we can use a tool like `Postman` to send `HTTP requests` to the endpoints. For example, to create a new document in the collection, we can send a **`POST`** request to http://localhost:5000/data with a JSON payload in the request body. Similarly, to retrieve all documents from the collection, we can send a **`GET`** request to http://localhost:5000/data. **`PUT`** and **`DELETE`** act the same.

## Executing queries

Now that we have created our database, collection, and API, we can use MongoDB's powerful query capabilities to retrieve and analyze the data in various ways. Here are some example queries:

In [21]:
import pymongo

# Establish a connection to the MongoDB database
client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client['titanic']
collection = db['passengers']

### 1. Get the number of passengers in each class

In [22]:
# Get the number of passengers in each class
results = collection.aggregate([
    {
        '$group': {
            '_id': '$class',
            'count': {'$sum': 1}
        }
    }
])
print('Number of passengers in each class: ', '\n')
for result in results:
    print(f'{result["_id"]} Class: {result["count"]}', '\n')


Number of passengers in each class: 

First Class: 216 

Second Class: 184 

Third Class: 487 



### 2. Get the average age of male and female passengers

In [23]:
# Get the average age of male and female passengers
results = collection.aggregate([
    {
        '$group': {
            '_id': '$sex',
            'average_age': {'$avg': {'$toDouble': '$age'}}
        }
    }
])
print('\nAverage age of male and female passengers: ', '\n')
for result in results:
    print(f'{result["_id"]}: {result["average_age"]:.2f}', '\n')


Average age of male and female passengers:  

female: 27.72 

male: 30.43 



### 3. Get the survival rate for each class

In [24]:
# Get the survival rate for each class
results = collection.aggregate([
    {
        '$group': {
            '_id': '$class',
            'survival_rate': {'$avg': {'$toInt': '$survived'}}
        }
    }
])
print('\nSurvival rate for each class: ', '\n')
for result in results:
    print(f'{result["_id"]} Class: {result["survival_rate"]:.2%}', '\n')


Survival rate for each class:  

First Class: 62.96%

Second Class: 47.28% 

Third Class: 24.24% 



### 4. Get the top 5 passengers (Man or Woman) with the highest fares

In [25]:
# Get the top 5 passengers with the highest fares
results = collection.find({}, {'_id': 0, 'who': 1, 'fare': 1}).sort('fare', pymongo.DESCENDING).limit(5)
print('\nPassengers with the highest fares:', '\n')
for result in results:
    print(f'{result["who"]}: ${float(result["fare"]):.2f}', '\n')


Passengers with the highest fares:  

Man: $512.33 

Man: $263.00 

Man: $263.00 

Woman: $262.38 

Man: $262.38 



### 5. Get the age and class of all female survivors under the age of 18

In [26]:
# Get the age and class of all female survivors under the age of 18
results = collection.find(
    {'sex': 'female', 'survived': True, 'age': {'$lt': '18'}},
    {'_id': 0, 'name': 1, 'age': 1}
)
print('\nFemale survivors under the age of 18: ', '\n')
for result in results:
    print(f'Age {float(result["age"])}, Class {result["pclass"]:.2f}', '\n')


Female survivors under the age of 18: 

Age 14.50, Class 2 

Age 14.50, Class 2 

Age 3.00, Class 3 

Age 5.00, Class 3 

Age 0.75, Class 1 

Age 4.00, Class 1 

Age 0.75, Class 1 

Age 5.00, Class 1 

Age 1.00, Class 2 

Age 0.75, Class 1 

Age 4.00, Class 1 

Age 16.00, Class 3 

Age 1.00, Class 2 

Age 16.00, Class 1 

Age 22.00, Class 2 

Age 14.00, Class 2 



In [27]:
# Close the connection to the MongoDB database
client.close()